In [70]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge,  ElasticNet

# Objectif

Je veux créer un modèle de Machine Learning à predire la consommation électrique prévue pour la semaine à venir.

La consommation électrique j'ai utilisé est `Global_active_power`.

# charger les données

In [71]:
day_df = pd.read_csv('../data/output/day_cleaned_household_power_consumption.csv', header=0, 
                    infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

# diviser un ensemble de données en ensembles train / validation

Je vais utiliser les données des trois premières années pour l'entrainement du modèle et la dernière année pour l'évaluation du modèle.

Je veux que les données d'un ensemble de données seront divisées en semaines. Ce sont des semaines qui commencent un dimanche et se terminent un samedi. Donc, je laisse tomber le premier jour qui est Samedi, et la dernière semaine sans Samedi.

In [72]:
def train_valid_diviser(values):
    '''
    diviser un ensemble de données en ensembles train / test par années
    
    Parameters:
    -- values - un np.array avec 2 dimensions
    '''
    # divisées par années
    train, valid = values[1:-328], values[-328:-6]
    return train, valid

In [73]:
train, valid = train_test_diviser(day_df.values)

Maintenant, je prends `Global_active_power` qui est dans la première colonne.

In [74]:
train_conso = train[:,0]
valid_conso = valid[:,0]

# créer le modèle

J'applique l'algo `Recursive Multi-Step Forecasting`. En gros, elle crée le modèle fait une prédiction pour un pas de temps, et elle prend cette prédiction comme les donées pour créer un nouveau model et faire la prochaine prédiction dans un pas de temps suivant. Ce processus est répété jusqu'à ce que le nombre d'étapes souhaité ait été prévu.

Ici, un pas de temps est une semaine. Le modèle est un modèle supervisé.

## transformer les données

Tout d'abord, il faut transformer les données dans deux parties, `feature` et `target`, pour entrainer le modèle. Dans Machine Learning, `feature` est les données pour entrainer le modèle, `target` est le données qu'on veut prédire.

Dans ce projet, `feature` est les consommations dans un pas de temps, `target` est la consommation dans le jour suivant. Par exemple, si je prends les consommations de Dimanche à Samedi comme `feature`, alors le prochain Dimanche est `target`.

In [75]:
def feature_target_split(data, stride = 7):
    '''
    transformer les donnéés dans 2 parties
    
    Parameters:
    -- data - la conso, 1 dimension
    -- slide - nombre de jour choisi pour créer feature. Une samaine (7) par defaut
    '''
    feature = []
    target = []
    
    i_start = 0
    for i in range(len(data)):
        i_end = i_start + stride
        if i_end < len(data):
            feature.append(data[i_start:i_end])
            target.append(data[i_end])
            i_start += 1
    
    return np.array(feature), np.array(target)

In [90]:
train_feature, train_target = feature_target_split(train_conso)

## construire le modèle pour un pas du temps

Dans cette partie, je simplement ajoute 2 couches avant le modèle de Machine Learning. Un couche est pour standardiser les données. La sortie est central réduit. La couche suivant est pour normaliser les données. Elle met les données entre 0 et 1.

Les modèles de machine learning que je choisis sont `Regression Lineair` et celles avec la penalité sur le nombre de `feature`, comme `Ridge`, `Lasso` et `ElasticNet`. Elles sont simple et interprétable.

In [106]:
def create_step_model(model):
    '''
    ajouter un couche de standardisation et un couche de normalisation avant le modèle de Machine Learning
    '''
    steps = []
    steps.append(('standardize', StandardScaler()))
    steps.append(('normalize', MinMaxScaler()))
    steps.append(('model', model))
    
    step_model = Pipeline(steps=steps)
    return step_model

## forcast

J'utilse le modèle que j'ai créé pour réaliser l'algo `Recursive Multi-Step Forecasting`

In [107]:
def recursive_multi_step_forecasting(model, feature):
    '''
    réaliser l'algo Recursive Multi-Step Forecasting
    
    Parameters:
    -- model - le modèle d'un pas du temps
    -- feature - les features qui creéent le modèle d'un pas du temps
    '''
    pred_sequence = []
    history = list(feature)
    stride = len(feature)
    
    for j in range(stride):
        X = np.array(history[-stride:]).reshape(1, stride)
        pred = model.predict(X)[0]
        pred_sequence.append(pred)
        # ajouter la prediction dans les données entrées
        history.append(pred)
    return np.array(pred_sequence)

## créer le modèle final

In [140]:
def model_prediction(model, train, valid, stride=7):
    history = list(train)
    pred = []
    
    for i in range(0,len(valid),7):
        train_x, train_y = feature_target_split(history)
        step_model = create_step_model(model)
        step_model.fit(train_x, train_y)
        pred_sequence = recursive_multi_step_forecasting(step_model, train_feature[-1, :])
        
        pred.append(pred_sequence)
        # get real observation and add to history for predicting the next day
        history.append(valid[i])
    
    return np.array(pred).flatten()

In [141]:
pred = model_prediction(LinearRegression(), train_conso, valid_conso)